In [200]:
import ops
from ops.imports_ipython import *

# runs example from repository directory
home = os.path.dirname(os.path.dirname(ops.__file__))
os.chdir(os.path.join(home, 'projects', 'steph'))
print(os.getcwd())

/Users/sasha/PycharmProjects/OpticalPooledScreens/projects/steph


In [201]:
WILDCARDS = dict(well=3, tile=1) # change these to change the well and tile that you want to analyze
CYCLES = 11 # number of cycles

# image processing thresholds and expected values
THRESHOLD_READS = 50
THRESHOLD_DAPI = 2000
THRESHOLD_CELL = 2500
NUCLEUS_AREA = 40, 400

SBS_CYCLES = range(1, CYCLES + 1)

# color of bases
# lut = "lookup table", used to map one color to another like a filter
LUTS = [
    ops.io.GRAY,
    ops.io.GREEN,
    ops.io.RED,
    ops.io.MAGENTA,
    ops.io.CYAN
]

# for formatting tif images when they are saved?
DISPLAY_RANGES = [
    [500, 15000],
    [100, 10000],
    [100, 20000],
    [100, 8000],
    [100, 6000]
]

In [202]:
barcodes = pd.read_csv('design.csv').drop(columns="sgRNA") # list of barcodes along with which gene they target
barcodes["barcode"] = barcodes["barcode"].apply(lambda x: x[:CYCLES])
barcode_set = set(barcodes["barcode"])

In [203]:
# find sbs images and print paths
search = f'data/10x_Cycle*_Well{WILDCARDS["well"]}_Point3_{str(WILDCARDS["tile"]).rjust(4, "0")}*.ome.tif'
input_files = natsorted(glob(search))
print(len(input_files))
print(input_files)
# used to format output filenames
description = {'mag': "10X", "well": WILDCARDS["well"], 'tile': WILDCARDS['tile'], 'subdir': f'process_ipynb/tile{WILDCARDS["tile"]}', 'ext': 'tif'}

11
['data/10x_Cycle1_Well3_Point3_0001_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0785.ome.tif', 'data/10x_Cycle2_Well3_Point3_0001_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0785.ome.tif', 'data/10x_Cycle3_Well3_Point3_0001_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0785.ome.tif', 'data/10x_Cycle4_Well3_Point3_0001_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0785.ome.tif', 'data/10x_Cycle5_Well3_Point3_0001_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0785.ome.tif', 'data/10x_Cycle6_Well3_Point3_0001_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0785.ome.tif', 'data/10x_Cycle7_Well3_Point3_0001_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0785.ome.tif', 'data/10x_Cycle8_Well3_Point3_0001_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0393.ome.tif', 'data/10x_Cycle9_Well3_Point3_0001_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0785.ome.tif', 'data/10x_Cycle10_Well3_Point3_0001_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0785.ome.tif', 'data/10x_Cycle11_Well3_Point3_0001_ChannelDAPI,G-ISS,T-ISS,A-ISS,C-ISS_Seq0785.ome.tif']


In [204]:
data = np.array([read(f) for f in input_files])

In [205]:
data, x_offsets, y_offsets = Snake._align_SBS(data, method="SBS_mean") # rigid alignment of sequencing cycles and channels.
save(name(description, tag='aligned'), data, display_ranges=DISPLAY_RANGES, luts=LUTS)

In [206]:
loged = Snake._transform_log(data, skip_index=0) # apply Laplacian-of-Gaussian filter from scipy.ndimage.
# save(name(description, tag='log'), loged, display_ranges=DISPLAY_RANGES, luts=LUTS)

In [207]:
maxed = Snake._max_filter(loged, 3, remove_index=0) # apply a maximum filter in a window of `width`. Conventionally operates on Laplacian-of-Gaussian filtered SBS data, dilating sequencing channels to compensate for single-pixel alignment error.
save(name(description, tag='maxed'), maxed, display_ranges=DISPLAY_RANGES[1:], luts=LUTS[1:])

In [208]:
std = Snake._compute_std(loged, remove_index=0) # use standard deviation over cycles, followed by mean across channels to estimate sequencing read locations.
# save(name(description, tag='std'), std)

In [209]:
peaks = Snake._find_peaks(std) # where are the spots
# save(name(description, tag='peaks'), peaks)

### segment nuclei and cells

In [210]:
# Find nuclei from DAPI (fluorescent stain)
# change first argument if DAPI staining is only done for a certain cycle, eg. 11th cycle would be data[10]
nuclei = Snake._segment_nuclei(data[0], THRESHOLD_DAPI,
 area_min=NUCLEUS_AREA[0], area_max=NUCLEUS_AREA[1])

save(name(description, tag='nuclei'), nuclei, compress=1)

In [211]:
cells = Snake._segment_cells(data[0], nuclei, THRESHOLD_CELL) # Matches cell labels to nuclei labels.
save(name(description, tag='cells'), cells, compress=1)

### extract base intensity, call reads, assign to cells

In [212]:
# Find the signal intensity from `maxed` at each point in `peaks` above `threshold_peaks`.
df_bases = Snake._extract_bases(maxed, peaks, cells,
                        THRESHOLD_READS, wildcards=WILDCARDS)
# df_bases.to_csv(name(description, tag='bases', ext='csv'), index=None)

In [213]:
df_reads = Snake._call_reads(df_bases, peaks=peaks) # call reads by compensating for channel cross-talk and calling the base with the highest corrected intensity for each cycle. Q = quality?
filename = name(description, tag='reads', ext='csv')
# df_reads.to_csv(filename, index=None)

In [214]:
# read from csv to match numerical precision of snakemake pipeline
df_reads = pd.read_csv(filename) 
df_cells = Snake._call_cells(df_reads) # gets the two most-common barcode reads for each cell.
df_cells.to_csv(name(description, tag='cells', ext='csv'), index=None)

### annotated SBS images

In [215]:
# last channel annotates base calls
annotate_luts = LUTS + [ops.annotate.GRMC, ops.io.GRAY]
annotate_display_ranges = [(a / 4, b / 4) for a,b in DISPLAY_RANGES] + [(0, 4)]
annotate_SBS = Snake._annotate_SBS(log=loged, df_reads=df_reads)

save(name(description, tag='annotate_SBS'), annotate_SBS,
     display_ranges=annotate_display_ranges, luts=annotate_luts, compress=1)

### further analysis :)

In [216]:
from collections import defaultdict

In [217]:
barcodes = defaultdict(int)

for index, row in df_cells.iterrows():
    b0 = row["cell_barcode_0"]
    b1 = row["cell_barcode_1"]
    # barcodes[b0] += row["cell_barcode_count_0"]
    # barcodes[b1] += int(row["cell_barcode_count_1"])
    if b0 in barcode_set: # lots of messed up reads...
        barcodes[b0] += row["cell_barcode_count_0"]
    if b1 in barcode_set:
        barcodes[b1] += int(row["cell_barcode_count_1"])

print(sorted(barcodes.items(), key=lambda x: x[1], reverse=True))

[('CTCAACCACTT', 9), ('TTAGCCGGATG', 9), ('GGGCCAAGAAT', 8), ('TCACAGTTTTC', 6), ('ACCATTAGGCC', 6), ('AACATGGCACT', 6), ('GAAGAAGGATG', 6), ('AGCTTACAATA', 6), ('TCTTTCCAACT', 5), ('GTTGCCAGCTT', 5), ('GACCCCTAGAT', 5), ('GGGCATGAGAT', 5), ('AGAGAATTCTG', 5), ('GCTAGTACCCA', 5), ('GTCCAATTTTC', 5), ('GCTTCAAGTAT', 4), ('AGCCTGACCTT', 4), ('GTGATTCCTTC', 4), ('ATAATAGCATT', 4), ('TCTAGCTGAGT', 3), ('TGATCTCCCAT', 3), ('CGCCTGCGCAC', 3), ('CTCCCCGACTA', 3), ('TAGTCTACATG', 3), ('GGTCGTGGGTC', 2), ('CCCACACCCCC', 2), ('GGGCTTCTGCT', 2), ('CCCACCACCCC', 2), ('TCTAATAGTAT', 2), ('ACTACAGTCCA', 2), ('CAGAGTAATAT', 2), ('CAGCAGAAGTG', 2), ('GGAATGGGAAG', 2), ('TTGACATTGCC', 2), ('TTCACCGTCCA', 2), ('CCAATCAGTGC', 2), ('TGTCGACAAAT', 2), ('GCTTCGAGGCT', 2), ('CGTCTGTTCAG', 2), ('CTCAAACTTAT', 2), ('GAACCACCACC', 2), ('CACGCCCCCCA', 1), ('GCATCACTAAA', 1), ('AGAAAAAATGT', 1), ('GCAACAGAGCC', 1), ('TCGCAAGGAAG', 1), ('GTTGGGTGGCA', 1), ('GATCTCAACTC', 1), ('AGCCCCTCCTT', 1), ('CCTCTCCCAAT', 1),